In [ ]:
# Mount google drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Necessary packages

import cv2
import numpy as np
#import matplotlib.pyplot as plt
import os
import csv
import pandas as pd
import torch
import torch.nn
import torch.optim
import torch.nn.functional

In [ ]:
# Global Vars

device = 'cuda:0'

desired_height = 512
desired_width = 256

b_size = 8
num_batches = 16

In [ ]:
# Preprocess data, generate slope file
def all_angles(img_dir, filenames_csv, landmarks_csv, out_dir):
    
    all_landmarks = pd.read_csv(landmarks_csv, header=None)
    filenames = pd.read_csv(filenames_csv, header=None)
    
    angles = []    
    for i, name in enumerate(filenames.iloc[:,0]):
        img = cv2.imread(img_dir + name)
        w = img.shape[0]
        h = img.shape[1]

        curr_landmark = all_landmarks.loc[i].values
        curr_slopes = []
        for m in range (0,68,4):
            slope_upper = (round(w * curr_landmark[m + 1 + 68]) - round(w * curr_landmark[m + 68])) / (
                     round(h * curr_landmark[m + 1]) - round(h * curr_landmark[m]) + 1e-7)
            slope_lower = (round(w * curr_landmark[m + 3 + 68]) - round(w * curr_landmark[m + 2 + 68])) / (
                    round(h * curr_landmark[m + 3]) - round(h * curr_landmark[m + 2]) + 1e-7)
            curr_slopes.append((slope_upper + slope_lower) / 2)
            
        curr_angles = []
        # 8 angles
        for i in range(1, 9):
          curr_angles.append(abs(np.rad2deg(np.arctan((
        curr_slopes[0] - curr_slopes[i]) / (1+curr_slopes[0] * curr_slopes[i])))) / 180)
        # 7 angles
        for i in range(9, 16):
          curr_angles.append(abs(np.rad2deg(np.arctan((
        curr_slopes[i] - curr_slopes[16]) / (1+curr_slopes[i] * curr_slopes[16])))) / 180)
        # 118 angles
        for i in range(1, 16):
          for j in range(i + 1, 16):
            curr_angles.append(abs(np.rad2deg(np.arctan((
        curr_slopes[i] - curr_slopes[j]) / (1+curr_slopes[i] * curr_slopes[j])))) / 180)
        angles.append(curr_angles)

    np.savetxt(out_dir, angles, delimiter =", ", fmt ='% s')

# train
all_angles('./drive/MyDrive/boostnet_labeldata/data/training/','./drive/MyDrive/boostnet_labeldata/labels/training/filenames.csv','./drive/MyDrive/boostnet_labeldata/labels/training/landmarks.csv', "./drive/MyDrive/boostnet_labeldata/training_set_all_angles.csv")
print("training set preprocessing complete !")

# test
all_angles('./drive/MyDrive/boostnet_labeldata/data/test/','./drive/MyDrive/boostnet_labeldata/full_test_filenames.csv','./drive/MyDrive/boostnet_labeldata/full_test_landmarks.csv', "./drive/MyDrive/boostnet_labeldata/test_set_all_angles.csv")
print("test set preprocessing complete !")

training set preprocessing complete !
test set preprocessing complete !


In [ ]:
# Load train, test data

# train dir
train_img = "./drive/MyDrive/boostnet_labeldata/data/training/"
train_filenames = csv.reader(open("./drive/MyDrive/boostnet_labeldata/labels/training/filenames.csv", 'r'))
train_landmarks = csv.reader(open("./drive/MyDrive/boostnet_labeldata/training_set_all_angles.csv", 'r'))

# test dir
test_img = "./drive/MyDrive/boostnet_labeldata/data/test/"
test_filenames = csv.reader(open("./drive/MyDrive/boostnet_labeldata/full_test_filenames.csv", 'r'))
test_landmarks = csv.reader(open("./drive/MyDrive/boostnet_labeldata/test_set_all_angles.csv", 'r'))

class creatDataset(torch.utils.data.Dataset):
    def __init__(self, img_dir, filenames_dir, landmarks_dir, is_train):
        self.img_dir = img_dir
        self.filenames = filenames_dir
        self.landmarks = landmarks_dir
        self.images = []
        self.labels = []
        
        for landmark in self.landmarks:
            coordinate_list = []
            for coordinate in landmark:
                coordinate_list.append(float(coordinate))
            self.labels.append(torch.Tensor(coordinate_list))

        for i, filename in enumerate(self.filenames):
            image = cv2.imread(self.img_dir + filename[0], cv2.IMREAD_GRAYSCALE)
            image = cv2.resize(image, (desired_width, desired_height))
            
            image = np.reshape(image, (1, image.shape[0], image.shape[1]))
            image_tensor = torch.from_numpy(image).float()
            
            self.images.append(image_tensor)

        if is_train:
          self.images = self.images[:-1]
          self.labels = self.labels[:-1]

    def __getitem__(self, index):
        image = self.images[index]
        label = self.labels[index]
        return image, label

    def __len__(self):
        return len(self.images)

# train
training = creatDataset(train_img, train_filenames, train_landmarks, True)
train_loader = torch.utils.data.DataLoader(training, batch_size=b_size, shuffle=True, num_workers=2)
print("training data loaded !")

# test
test = creatDataset(test_img, test_filenames, test_landmarks, False)
test_loader = torch.utils.data.DataLoader(test, batch_size=b_size, shuffle=True, num_workers=2)
print("test data loaded !")

training data loaded !
test data loaded !


In [ ]:
# Loss function

#loss_fcn = torch.nn.CrossEntropyLoss().to(device)
loss_fcn = torch.nn.MSELoss().to(device)

def compute_loss(net, data_loader):
    s = 0
    with torch.no_grad():
        for i, data in enumerate(data_loader, 0):
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            prediction = net(images)
            loss = loss_fcn(prediction.float(), labels.float())
            s += loss.item()
            
    return s / len(data_loader)

# Network

class Network(torch.nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.pool_layer = torch.nn.MaxPool2d(2, 2)
        self.con_layer_1 = torch.nn.Conv2d(1, 2, 3, 1, 1)
        self.con_layer_2 = torch.nn.Conv2d(2, 4, 3, 1, 1)
        self.con_layer_3 = torch.nn.Conv2d(4, 8, 3, 1, 1)
        self.con_layer_4 = torch.nn.Conv2d(8, 16, 3, 1, 1)
        self.con_layer_5 = torch.nn.Conv2d(16, 16, 3, 1, 1)
        self.fully_connected_1 = torch.nn.Linear(16 * 16 * 8, 512)
        self.fully_connected_2 = torch.nn.Linear(512, 120)
        
    def forward(self, x):        
        x = torch.nn.functional.relu(self.con_layer_1(x))
        x = self.pool_layer(x)
        x = torch.nn.functional.relu(self.con_layer_2(x))
        x = self.pool_layer(x)
        x = torch.nn.functional.relu(self.con_layer_3(x))
        x = self.pool_layer(x)
        x = torch.nn.functional.relu(self.con_layer_4(x))
        x = self.pool_layer(x)
        x = torch.nn.functional.relu(self.con_layer_5(x))
        x = self.pool_layer(x)
        x = x.view(-1, 16 * 16 * 8)
        x = torch.nn.functional.relu(self.fully_connected_1(x))
        x = self.fully_connected_2(x)
        return x

net = Network().to(device)

In [ ]:
# Training

optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=200, gamma=0.9)

epoch = 200

for i in range(epoch):
    for j, data in enumerate(train_loader, 0):
                
        inputs = data[0].to(device)
        labels = data[1].to(device)
    
        optimizer.zero_grad()

        predictions = net(inputs)
        loss = loss_fcn(predictions.float(), labels.float())
        loss.backward()
        optimizer.step()
        
    scheduler.step()    
    print("Iteration:", i + 1, "train_loss", compute_loss(net, train_loader), "test_loss", compute_loss(net, test_loader))

print('Training Completed !')

Iteration: 1 train_loss 0.004214394182781689 test_loss 0.003726842758624116
Iteration: 2 train_loss 0.003422327912024533 test_loss 0.003221741582819959
Iteration: 3 train_loss 0.0036005240040443216 test_loss 0.0035952509060734883
Iteration: 4 train_loss 0.002492892546191191 test_loss 0.0029198918491601944
Iteration: 5 train_loss 0.0026707007525449927 test_loss 0.0033455698248872068
Iteration: 6 train_loss 0.001940727940624735 test_loss 0.0029912393947597593
Iteration: 7 train_loss 0.001725759500307807 test_loss 0.0026671239429560956
Iteration: 8 train_loss 0.0015066971541576398 test_loss 0.002607074664410902
Iteration: 9 train_loss 0.0013739122567737164 test_loss 0.0027358766819816083
Iteration: 10 train_loss 0.0014006582355553594 test_loss 0.0027826081204693764
Iteration: 11 train_loss 0.001321930766183262 test_loss 0.0027567813303903677
Iteration: 12 train_loss 0.0013903203992716346 test_loss 0.002937498895335011
Iteration: 13 train_loss 0.001367527785381147 test_loss 0.0031403120301

In [ ]:
# Save model and results

torch.save(net.state_dict(), "./drive/MyDrive/boostnet_labeldata/model_2/torch_model_2")

test_result = []
dataiter = iter(test_loader)
for i in range(num_batches):
  image, label = dataiter.next()
  prediction = net(image.to(device))
  for j in range(b_size):
    test_result.append(prediction[j].tolist())
np.savetxt("./drive/MyDrive/boostnet_labeldata/model_2/test_all_angles.csv", test_result, delimiter =", ", fmt ='% s')

In [ ]:
# Compute angles and SMAPE

def cobb_angles(img_dir, filenames_csv, all_angles_csv):

    # get current model prediction
    all_angles = pd.read_csv(all_angles_csv, header= None)
    filenames = pd.read_csv(filenames_csv, header= None)
    
    angles = []
    for i, names in enumerate(filenames.iloc[:,0]):

        curr_angles = all_angles.loc[i].values

        pt_lower_v_index = np.argmax(curr_angles[:8])
        pt = np.max(curr_angles[:8])

        tl_upper_v_index = np.argmax(curr_angles[8:15]) + 8
        tl = np.max(curr_angles[8:15])

        mt_index = 15
        for i in range(pt_lower_v_index):
          mt_index += 14 - i
        mt_index += tl_upper_v_index - pt_lower_v_index

        mt = curr_angles[mt_index]

        angles.append([pt * 180, mt * 180, tl * 180])

        
    # GT
    angle_gT = list(csv.reader(open("./drive/MyDrive/boostnet_labeldata/full_test_angles.csv", 'r')))
    for i in range(len(angle_gT)):
      for j in range(3):
        angle_gT[i][j] = float(angle_gT[i][j])


    # print
    less_than_five = 0
    five_to_ten = 0
    ten_to_twenty = 0
    twenty_above = 0

    smape_below_5_percent = 0
    smape_below_10_percent = 0
    smape_above_10_percent = 0
    smape_above_20_percent = 0
    

    smape = 0
    avg_difference = []
    for i in range(128):
      print(str(i + 1) + " prediction: " + str(angles[i]))
      print(str(i + 1) + " groundtruth: " + str(angle_gT[i]))
      print(str(i + 1) + " difference: " + str(np.abs(np.asarray(angle_gT[i]) - np.asarray(angles[i]))))
      num = 0
      denom = 0
      for j in range(3):
        diff = np.abs(angle_gT[i][j] - angles[i][j])
        num += diff

        if diff <= 5:
          less_than_five += 1
        elif 5 < diff and diff <= 10:
          five_to_ten += 1
        elif 10 < diff and diff <= 20:
          ten_to_twenty += 1
        elif 20 < diff:
          twenty_above += 1

        denom += angle_gT[i][j] + angles[i][j]
      smape += (num / denom)
      avg_difference.append(np.sum(np.abs(np.asarray(angle_gT[i]) - np.asarray(angles[i]))))

      sam = num / denom
      if sam >= 0.2:
        smape_above_20_percent += 1
      elif sam >= 0.1:
        smape_above_10_percent += 1
      elif sam >= 0.05:
        smape_below_10_percent += 1
      else:
        smape_below_5_percent += 1
      print("SMAPE: " + str((num / denom)))
      print('\n')
      # plt.plot([i for i in range(len(confidence[i]))], confidence[i])
      # plt.show()
    smape /= 128

    print("SMAPE: " + str(smape))

    print("Difference less than 5 degree: " + str(less_than_five / 384))
    print("Difference between 5 and 10 degree: " + str(five_to_ten / 384))
    print("Difference between 10 and 20 degree: " + str(ten_to_twenty / 384))
    print("Difference above 20 degree: " + str(twenty_above / 384))
    print("\n")
    print("Difference less than 5 degree: " + str(less_than_five))
    print("Difference between 5 and 10 degree: " + str(five_to_ten))
    print("Difference between 10 and 20 degree: " + str(ten_to_twenty))
    print("Difference above 20 degree: " + str(twenty_above))
    print("\n")
    print("# SMAPE less than 5%: " + str(smape_below_5_percent))
    print("# SMAPE between 5% and 10%: " + str(smape_below_10_percent))
    print("# SMAPE between 10% and 20%: " + str(smape_above_10_percent))
    print("# SMAPE above 20%: " + str(smape_above_20_percent))

cobb_angles('./drive/MyDrive/boostnet_labeldata/data/test/','./drive/MyDrive/boostnet_labeldata/full_test_filenames.csv','./drive/MyDrive/boostnet_labeldata/model_2/test_all_angles.csv')

1 prediction: [6.255367398262008, 9.251432418823235, 10.682581365108486]
1 groundtruth: [43.423, 15.872, 40.687]
1 difference: [37.1676326   6.62056758 30.00441863]
SMAPE: 0.5848601967132975


2 prediction: [10.86186826229094, 31.001438498497006, 18.002442419528958]
2 groundtruth: [34.609, 33.592, 12.048]
2 difference: [23.74713174  2.5905615   5.95444242]
SMAPE: 0.23046921075513266


3 prediction: [7.405831217765808, 23.282840251922597, 11.662075817584991]
3 groundtruth: [26.437, 15.781, 8.0784]
3 difference: [19.03116878  7.50184025  3.58367582]
SMAPE: 0.325068668961368


4 prediction: [25.14207512140274, 45.31782567501068, 28.88578176498412]
4 groundtruth: [48.061, 29.951, 19.867]
4 difference: [22.91892488 15.36682568  9.01878176]
SMAPE: 0.23985097455469753


5 prediction: [17.0439738035202, 11.180753409862511, 21.227356195449822]
5 groundtruth: [41.524, 19.011, 23.36]
5 difference: [24.4800262   7.83024659  2.1326438 ]
SMAPE: 0.2582952375911213


6 prediction: [18.25922772288322, 